In [1]:
import pandas as pd
import neptunecontrib.monitoring.optuna as optuna_utils
import optuna
import numpy as np
import neptune
import gc


import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier


from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

In [2]:
## инициализируем проект в нептуне
neptune.init('iliaavilov/Zindi-insurance')

Project(iliaavilov/Zindi-insurance)

In [3]:
random_state = 555

# Загрузка данных

In [4]:
# Тренировочные данные
train = pd.read_csv('train_prepared.csv')

In [5]:
def encoding(data, column):
    
    """
    Кодировка колонки датафрейма через LabelEncoder из sklearn

    ----------
    data_train - датафрейм
    column - колонка датафрейма, которую надо закодировать
    ----------

    """
    
    le = LabelEncoder()
    le.fit(data[column].values)
    data[column] = le.transform(data[column].values)
    
    return data

In [6]:
train = encoding(train, 'variable')

# Кросс валидация

In [7]:
users = train[['ID']].drop_duplicates().reset_index(drop = True)

In [8]:
def split_cv(data, n_splits, random_state):
    
    """
    Разбивает датафрейм на несколько фолдов CV с помощью KFold из sklearn

    ----------
    data - датафрейм
    n_splits - количество фолдов в CV
    random_state - сид рандомного состояния для KFold из sklearn
    ----------

    """    
    
    kf = KFold(n_splits = n_splits, shuffle = True, random_state = random_state)
    cv = list(kf.split(data))
    cv_users = []
    
    for cur_cv in cv:
        train = data.iloc[cur_cv[0]]['ID'].values
        test = data.iloc[cur_cv[1]]['ID'].values
        cv_users.append((train, test))
        
    return(cv_users)

In [11]:
cv_users = split_cv(users, n_splits= 3, random_state = random_state)

In [12]:
cv = []

for fold in cv_users:
    cv.append((
        train[train['ID'].isin(fold[0])].index.values, 
        train[train['ID'].isin(fold[1])].index.values))

In [13]:
train = train.drop('ID', axis = 'columns')

# Подбор модели

### Тренировка разных моделей

In [14]:
def lgbm_training(X_train, y_train, X_test, y_test, params):
    
    """
    Тренирует lgbm модель и возвращает log loss модели на тестовой выборке

    ----------
    X_train - тренировочные данные
    y_train - тренировочный target
    X_test - тестовые данные
    y_test - тестовый target
    params - параметры модели
    ----------

    """        

    train_data = lgb.Dataset(X_train, y_train)
    model_trial = lgb.train(params, train_data)
    predictions = model_trial.predict(X_test)
    cur_score = log_loss(y_test, np.array([1- predictions, predictions]).T)
    
    return(cur_score)



def xgb_training(X_train, y_train, X_test, y_test, params):

    """
    Тренирует xgboost модель и возвращает log loss модели на тестовой выборке

    ----------
    X_train - тренировочные данные
    y_train - тренировочный target
    X_test - тестовые данные
    y_test - тестовый target
    params - параметры модели
    ----------

    """  
    
    train_data = xgb.DMatrix(X_train, label = y_train)
    test_data = xgb.DMatrix(X_test)
    model_trial = xgb.train(params = params, dtrain = train_data, num_boost_round = params['num_boost_round'])
    predictions = model_trial.predict(test_data)
    cur_score = log_loss(y_test, np.array([1- predictions, predictions]).T)
    
    return(cur_score)



def sklearn_models_training(X_train, y_train, X_test, y_test, params):

    """
    Тренирует sklearn модель (или любую другую модель с таким же функционалом) и возвращает log loss 
    модели на тестовой выборке

    ----------
    X_train - тренировочные данные
    y_train - тренировочный target
    X_test - тестовые данные
    y_test - тестовый target
    params - параметры модели
    ----------

    """  
    
    model_trial = clone(model)
    model_trial.set_params(**params)
    model_trial.fit(X_train, y_train)
    predictions = model_trial.predict_proba(X_test)
    cur_score = log_loss(y_test, y_train)
    
    return(cur_score)

### Функция, которую optuna будет пытаться минимизировать

In [15]:
def objective(trial, model, train, cv, random_state):

    """
    Objective функция для study из optuna. Возвращает очки на каждом этапе подбора гиперпараметров

    ----------
    trial - объект обучения из optuna
    model - модель (xgboost/lgbm/sklean-подобная модель)
    train - тренировочный датасет и таргет
    random_state - сид рандомного состояния для KFold из sklearn
    ----------

    """  
    
    ## Множество параметров моделей
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'n_jobs': -1,
        'n_estimators': trial.suggest_int('n_estimators', 500, 2500),
        'random_state': random_state,
        'categorical_feature': [train.columns.get_loc(cat_col) for cat_col in 
                                ['sex', 'marital_status', 'branch_code', 'occupation_code',
                                 'occupation_category_code', 'variable', 'P5DA',
                                 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9',
                                 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW',
                                 'GHYX', 'ECY3']],
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0)
    }
    
    ## Список результатов для разных фолдов
    score = []
    
    for fold in cv:
        
        ## Выбираем X и y
        X_train = train.iloc[fold[0], :].drop('presence', axis = 'columns')
        y_train = train.iloc[fold[0], :]['presence'].values
        X_test = train.iloc[fold[1], :].drop('presence', axis = 'columns')
        y_test = train.iloc[fold[1], :]['presence'].values
        
        
        ## В зависимости от заданной модели трренируем соответствующую модель
        if model == 'lgbm':
            cur_score = lgbm_training(X_train, y_train, X_test, y_test, params)
            score.append(cur_score)
            
        elif model == 'xgb':
            cur_score = xgb_training(X_train, y_train, X_test, y_test, params)
            score.append(cur_score)
            
        else:
            cur_score = sklearn_models_training(X_train, y_train, X_test, y_test, params)
            score.append(cur_score)
        
        
        print(cur_score)
    
    ## Логируем список результатов для каждого фолда
    neptune.log_text('CV scores', str(score))
    
    ## Возвращаем средний результат по фолдам, который будем минимизировать
    return(np.mean(score))

In [16]:
def training(train, cv, model, random_state, n_trials, tags):

    """
    Подбирает гиперрпараметры для заданной модели

    ----------
    train - тренировочный датасет и таргет
    cv - список списков с индексами для трренировчной/тестовой части вида [[train_index0, test_index0], ...]
    model - модель (xgboost/lgbm/sklean-подобная модель)
    random_state - сид рандомного состояния для KFold из sklearn
    n_trials - количество попыток найти лучшие гиперпараметры
    tags - тэги для записи эксперимента в optuna
    ----------

    """  
    
    ## Делаем переменную study глобальной, чтобы в случае незапланированной остановки можно было продолжить подбор 
    ## параметров с того момента, на котором остановились
    global study
    
    ## Создаем эксперимент в neptune
    if type(model) == str:
        current_experiment = neptune.create_experiment(model, tags = tags)
    else:
        current_experiment = neptune.create_experiment(type(model).__name__, tags = tags)
        
    ## Выбираем стандартный сэмплер для подборки параметров
    sampler = optuna.samplers.TPESampler(seed = random_state)
    
    ## Задаем объект study, с помощью которого будем МИНИМИЗИРОВАТЬ ошибку
    study = optuna.create_study(sampler = sampler, direction = 'minimize')
    
    ## минимизируем ошибку
    study.optimize(lambda trial: objective(trial, model, train, cv, random_state), 
                   n_trials = n_trials, callbacks = [optuna_utils.NeptuneCallback()]
                  )
    
    ## Логируем объект study
    optuna_utils.log_study(study)
    
    neptune.stop()

In [ ]:
training(train, 
         cv, 
         model = 'lgbm', 
         random_state = random_state, 
         n_trials = 150, 
         tags = ['catfeatures', 'lgbm catfeatures', 'timestamp from registartion date', 
                 'all initial features', 'all initial data', 'dropped_nans', 'one model', 'cv ver.2'])

https://ui.neptune.ai/iliaavilov/Zindi-insurance/e/ZIN-922
0.03200842000695025
0.03277893643365938
0.03168116624582877


[I 2020-09-07 21:57:56,365] Finished trial#0 with value: 0.0321561742288128 with parameters: {'n_estimators': 910, 'num_leaves': 176, 'learning_rate': 0.012709753733202291, 'min_child_samples': 70, 'feature_fraction': 0.9953822181510033, 'bagging_fraction': 0.6832014169448941}. Best is trial#0 with value: 0.0321561742288128.


0.05576140282553469
0.056906177909049366
0.052155166756764605


[I 2020-09-07 22:02:45,099] Finished trial#1 with value: 0.05494091583044955 with parameters: {'n_estimators': 1521, 'num_leaves': 242, 'learning_rate': 0.2980638271480203, 'min_child_samples': 86, 'feature_fraction': 0.6039943519772417, 'bagging_fraction': 0.7722612748995099}. Best is trial#0 with value: 0.0321561742288128.


0.033860309503940755
